ECOS TESS processing follows the same pattern as all of the other SppIn information gatherers. Running locally, I pull all messages and then process them in parallel at a rate that should not break anything. When running in a lambda environment, we will need to similarly throttle the number of concurrent connections we send to the ECOS API.

This process requires a local token variable to be set as it uses an API key connection to the IUCN Red List API. 

In [1]:
import pysgcn
sgcn = pysgcn.sgcn.Sgcn()

from joblib import Parallel, delayed
from tqdm import tqdm

mq = "mq_ecos_check"
sppin_source = "ecos"

In [2]:
messages = sgcn.sql_mq.get_all_records("mq", mq)
print(len(messages))
messages[99]

37617


{'id': '0a39a3a7683b4511e9c770818484f915f3beba8f',
 'date_inserted': '2019-12-19T13:22:10.017916',
 'body': {'source': {'type': 'List of Scientific Names',
   'name_source': 'ITIS Search'},
  'sppin_key': 'Scientific Name:Pseudotriton montanus flavissimus'}}

In [ ]:
%%time
Parallel(n_jobs=8, prefer="threads")(
    delayed(sgcn.process_sppin_source_search_term)
    (
        message_queue=mq,
        sppin_source=sppin_source,
        message_id=message["id"], 
        message_body=message["body"]
    ) for message in tqdm(messages)
)